In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

In [2]:
embedding=OllamaEmbeddings(model='mxbai-embed-large')
db=FAISS.load_local(folder_path="./data",index_name="vector_store",embeddings=embedding,allow_dangerous_deserialization=True)
llm = Ollama(
    model="phi3:mini"
) 

To stream the chunks when they arrive to avoid latency

In [3]:
query = "Tell me a joke"
for chunks in llm.stream(query):
    print(chunks)

 Why
 don
'
t
 scient
ists
 trust
 atoms
?
 Because
 they
 make
 up
 everything
!



In [4]:
llm.invoke('tell me a joke')

" Why don't scientists trust atoms?\n\n\nBecause they make up everything!"

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
            <|system|>
            You are a helpful assistant.<|end|>
            <|user|>
            Answer the following question based on the given context in 100-150 words:
            {context}
            Question:
            {input}
            <|end|>
            <|assistant|>

"""
)

In [6]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

docs_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, docs_chain)


In [7]:
response = retrieval_chain.invoke({"input": "Is this food product Healthy if its ingredients are: Refined Sunflower Oil \n and its Nutritional Information is: Energy: 900 Kcal, Protein: 0g, Carbohydrates: 0g, Fat: 100g, Saturated Fat: 10g,"})
print(response['answer'])

 This food product does not seem healthy based on the provided information. Although it contains refined sunflower oil as its main fat source instead of unhealthier saturated or trans fats commonly found in processed products, there are several issues to consider: 
1. It has no protein (0g), which is essential for a balanced diet and helps build and repair body tissues. The human body requires adequate amounts of high-quality proteins from various sources like lean meats, fish, legumes, nuts, and dairy products to maintain optimal health. 
2. It has zero carbohydrates (0g), which is also concerning as our bodies rely on this macronutrient for energy. Carbs are found in foods such as fruits, vegetables, whole grains, legumes, and dairy products. A diet devoid of these healthy sources could lead to nutritional deficiencies over time if consumed regularly or extensively without proper compensation from other macronutrients (proteins & fats). 
3. It has a high fat content of about 100g per

In [10]:
print(response)

{'input': 'Is this food product Healthy if its ingredients are: Refined Sunflower Oil \n and its Nutritional Information is: Energy: 900 Kcal, Protein: 0g, Carbohydrates: 0g, Fat: 100g, Saturated Fat: 10g,', 'context': [Document(metadata={'source': 'https://www.nhs.uk/live-well/eat-well/food-guidelines-and-food-labels/how-to-read-food-labels/'}, page_content='to as calories.They must also include information on fat, saturates (saturated fat), carbohydrate, sugars, protein and salt. They may also include other nutrients, like fibre.All nutrition information is provided per 100 grams or per 100 millilitres and sometimes per portion of the food or'), Document(metadata={'source': 'https://www.acko.com/health-insurance/how-to-read-food-labels-for-nutritional-choices/'}, page_content='saturated and trans fats, carbohydrates, fibre, sugar and vitamins and minerals.How do I know if a food item contains unhealthy trans fats?If a food label claims to contain no trans fats, check the ingredients 

In [17]:
output={'answer':response['answer'],'sources':[]}
for docs in response['context']:
    output['sources'].append(docs.dict()['metadata']['source'])
print(output)

{'answer': " This food product does not seem healthy based on the provided information. Although it contains refined sunflower oil as its main fat source instead of unhealthier saturated or trans fats commonly found in processed products, there are several issues to consider: \n1. It has no protein (0g), which is essential for a balanced diet and helps build and repair body tissues. The human body requires adequate amounts of high-quality proteins from various sources like lean meats, fish, legumes, nuts, and dairy products to maintain optimal health. \n2. It has zero carbohydrates (0g), which is also concerning as our bodies rely on this macronutrient for energy. Carbs are found in foods such as fruits, vegetables, whole grains, legumes, and dairy products. A diet devoid of these healthy sources could lead to nutritional deficiencies over time if consumed regularly or extensively without proper compensation from other macronutrients (proteins & fats). \n3. It has a high fat content of

In [25]:
llm.invoke(''''<|system|>
            You are a helpful assistant.<|end|>
            <|user|>
            Answer the following question in 100-150 words:
            Question:
            Is this food product Healthy if its ingredients are: Refined Sunflower Oil \n and its Nutritional Information is: Energy: 900 Kcal, Protein: 0g, Carbohydrates: 0g, Fat: 100g, Saturated Fat: 10g
            <|end|>
            <|assistant|>''')

" The provided food product appears to have a high energy density with only fat content and no significant amounts of protein or carbohydrates. Given that refined sunflower oil is mainly composed of unsaturated fats—which are considered healthier options in comparison to saturated fats found in animal products—it could be argued from this perspective alone that the product might have a place on your diet as part of moderation and balance with other foods. However, its high caloric content relative to zero protein or carbohydrate intake makes it an energy-dense option rather than healthy in terms of overall nutritional benefits for most people's needs.\n\nMoreover, the absence of fiber (assuming no other ingredients are present) is a downside since dietary fibers contribute to digestive health and satiety without adding calories or fat directly into our bodies—a factor often overlooked when considering nutritional value. High intake levels might also lead to increased energy consumption

In [26]:
llm.invoke('''Out of the following answers below which one is the better answer to the query: "Is this food product Healthy if its ingredients are: Refined Sunflower Oil \n and its Nutritional Information is: Energy: 900 Kcal, Protein: 0g, Carbohydrates: 0g, Fat: 100g, Saturated Fat: 10g"?
ANSWER1:This food product does not appear to be a healthy choice based on the given ingredients and nutritional information. Although it contains sunflower oil as an initial source of fat, which is generally considered better than other oils due to its unsaturated fats composition, the fact that 100g of this product provides you with such a high amount (90% or more) calories from fat alone suggests imbalance in your diet. Moreover, no protein and carbohydrates are present, which indicates lacking essential nutrients typically found in food products for balanced eating – proteins provide energy as well as critical roles in repair and growth of tissues; while complex carbs supply the body with a steady release of energy throughout. The product also contains 10g or about 25% saturated fats, which are known to raise cholesterol levels potentially leading towards heart diseases when consumed excessively over time. Also note that no details on dietary fibre and other nutrients like sugars is provided but high fat content alone suggests this isn't the best choice for a healthy balanced meal or snack, especially if eaten frequently due to its calorie-dense nature (90% of energy from fats). Always remember that while checking labels can give you an idea about what nutrients are included in your food choices and their amounts per 100g/ml serving size, it's important also consider the overall diet context. Moderation is key to a healthy eating pattern even with seemingly better-quality ingredients like sunflower oil here mentioned but without balanced nutrients intake in general and other food items consumed throughout the day or meal plan period.
ANSWER2:The provided food product appears to have a high energy density with only fat content and no significant amounts of protein or carbohydrates. Given that refined sunflower oil is mainly composed of unsaturated fats—which are considered healthier options in comparison to saturated fats found in animal products—it could be argued from this perspective alone that the product might have a place on your diet as part of moderation and balance with other foods. However, its high caloric content relative to zero protein or carbohydrate intake makes it an energy-dense option rather than healthy in terms of overall nutritional benefits for most people's needs.\n\nMoreover, the absence of fiber (assuming no other ingredients are present) is a downside since dietary fibers contribute to digestive health and satiety without adding calories or fat directly into our bodies—a factor often overlooked when considering nutritional value. High intake levels might also lead to increased energy consumption, potentially leading to weight gain if not balanced with physical activity for some individuals who are sedentary.\n\nConclusively, while refined sunflower oil has its merits as a source of unsaturated fats when consumed in moderation and alongside other nutrient-rich foods, this particular product's composition suggests it should not be classified as healthy due to the high fat content with no added benefits from proteins or carbohydrates. It is always crucial to evaluate such products within a broader dietary context and personalized nutritional requirements for overall well-being, considering factors like age, sex, physical activity levels, health status, etc.
''')

' ANSWER1 provides the most comprehensive evaluation of whether this food product can be considered "healthy" based on its ingredients (Refined Sunflower Oil) and nutritional information: Energy 900 Kcal; Protein 0g; Carbohydrates 0g; Fat 100g, Saturated Fat 10g. Although sunflower oil is a better fat source compared to saturated fats found in animal products, the fact that this product consists of such high amounts (96%) calories from fat alone with no presence of essential nutrients like proteins or carbohydrates raises concerns regarding its contribution towards balanced and healthy eating. It\'s critical to remember a well-balanced diet includes various macronutrients, not just fats for energy provision but also fibers (carbs), and essential amino acids found in proteins as they contribute significantly beyond mere caloric intake towards overall nutrition—repair, growth of tissues etc. Absence of these crucial components along with high saturated fat content suggest that this produ